In [ ]:
import pandas as pd

TASKNAMES = ["subtask1", "subtask2"]
TYPES = ["test", "test_preprocess"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for type in TYPES:
        print(f"Processing {taskname} for {type}")
        prefix = "dataset"
        # Load the data
        
        if taskname == "subtask2" and type == "test_preprocess":
            continue
        data = pd.read_csv(f"{prefix}/{taskname}_{type}.csv")
        dataset_loader[f"{taskname}_{type}"] = data

dataset_loader["subtask1_test"]["claim"] = dataset_loader["subtask1_test_preprocess"]["claim"]

In [ ]:
from nltk.tokenize import sent_tokenize
from rapidfuzz import fuzz
from utils import get_latest_result_by_prefix

from kg_compare import get_src_tgt_dict, get_map_src_tgt_dict, compare_knowledge_graphs_similarity
test_data = dataset_loader["subtask1_test"]

index = 6
claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_{index}_")
reference_nodes, reference_edges = get_latest_result_by_prefix(f"reference_test_{index}_")
reference_content = test_data.at[index, "reference"]

claim_node_list = list(claim_nodes.keys())
reference_node_list = list(reference_nodes.keys())

if claim_node_list == [] or reference_node_list == []:
    print("Claim or reference is empty")
    
    
similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                        reference_node_list,
                                                        if_plot=False)
src_tgt_dict = get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list)

claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)

def find_best_span_for_all_evidences(reference_edges,
                                     claim_reference_edge_map,
                                     paragraph,
                                     max_window_size=5):
    sentences = sent_tokenize(paragraph)

    best_span_list = []
    for _, reference_edge_list in claim_reference_edge_map.items():
        evidence_edges = set(reference_edge_list) & set(reference_edges.keys())

        for evidence_edge in evidence_edges:
            reference_edge = reference_edges[evidence_edge]

            best_score = -1
            best_span = None

            for window_size in range(1, min(max_window_size, len(sentences)) + 1):
                for i in range(len(sentences) - window_size + 1):
                    window = ' '.join(sentences[i:i+window_size])
                    score = fuzz.ratio(reference_edge[0]["description"], window)
                    if score > best_score:
                        best_score = score
                        best_span = (i, i+window_size)
            best_span_list.append(best_span)


    intervals = sorted(set(best_span_list), key=lambda x: x[0])
    merged = []
    for interval in intervals:
        if not merged or merged[-1][1] < interval[0]:
            merged.append(interval)
        else:
            merged[-1] = (merged[-1][0], max(merged[-1][1], interval[1]))

    print(merged)
    evidence_sentences = [' '.join(sentences[merged[i][0]:merged[i][1]]) for i in range(len(merged))]
    return ' '.join(evidence_sentences)


find_best_span_for_all_evidences(reference_edges,
                                 claim_reference_edge_map,
                                 reference_content)


In [ ]:
intervals = [
    (1, 2), (9, 10), (13, 14), (15, 16), (13, 14),
    (1, 2), (9, 10), (13, 14), (15, 16), (13, 14),
    (1, 2), (9, 10), (13, 14), (15, 16), (13, 14),
    (5, 6), (9, 10)
]

# Step 1: Sort and remove duplicates
intervals = sorted(set(intervals), key=lambda x: x[0])

# Step 2: Merge intervals
merged = []
for interval in intervals:
    if not merged or merged[-1][1] < interval[0]:
        merged.append(interval)
    else:
        merged[-1] = (merged[-1][0], max(merged[-1][1], interval[1]))

print(merged)

In [ ]:
claim_reference_edge_map

In [ ]:
import pandas as pd

TASKNAMES = ["subtask1", "subtask2"]
BATCHNAMES = ["batch1", "batch2", "batch3"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for batchname in BATCHNAMES:
        print(f"Processing {taskname} for {batchname}")
        prefix = "dataset/"
        # Load the data
        data = pd.read_csv(f"{prefix}/{taskname}_train_{batchname}.csv")
        dataset_loader[f"{taskname}_{batchname}"] = data

In [ ]:
print(dataset_loader["subtask1_batch2"].answer.tolist()[0])

In [ ]:
print(dataset_loader["subtask1_batch2"].claim.tolist()[0])

In [ ]:
nodes_reference, edges_reference = read_knowledge_graph_from_pickle(files[0])
nodes_claim, edges_claim = read_knowledge_graph_from_pickle(files[1])
for key in nodes_claim.keys():
    if key in nodes_reference.keys():
        print(f"`{key}`")
    else:
        print(f"Key `{key}` not found in nodes_reference")

In [ ]:
for key_src, key_tgt in edges_claim.keys():
    print(f"`{key_src}` -> `{key_tgt}`")

In [1]:
from geminillm import gemini_complete_if_cache
from json_kv_iml import JsonKVStorage
from operationCheatSheet import preprocessing_claim
from openaillm import openai_embed

LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"


kv_global_config = {
    "working_dir": "/tmp",
    "llm_model_name": LLM_MODEL_NAME,
    "embedding_batch_num": 64,  # or another integer suitable for your setup
    "vector_db_storage_cls_kwargs": {
        "cosine_better_than_threshold": 0.2  # or another float threshold you want
    },
    "base_url": "https://api.openai.com/v1",
}

llm_cache = JsonKVStorage(namespace="llm_cache", global_config=kv_global_config, embedding_func=openai_embed)
    
async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
    if history_messages is None:
        history_messages = []

    # Use Google GenAI
    return await gemini_complete_if_cache(
        model=LLM_MODEL_NAME,
        prompt=prompt,
        history_messages=history_messages,
        hashing_kv=llm_cache,
        temperature=0.2,
        max_tokens=max_tokens or 1024,
    )
# answer = dataset_loader["subtask1_batch2"].answer.tolist()[0]
# claim = dataset_loader["subtask1_batch2"].claim.tolist()[0]
# response = await preprocessing_claim(claim, answer, llm_wrapper, llm_cache, 1024, [])
# response

In [ ]:
all_answer_batch_2 = dataset_loader["subtask1_batch2"].answer.tolist()
all_claim_batch_2 = dataset_loader["subtask1_batch2"].claim.tolist()

preprocess_path = "dataset/subtask1_train_batch2_preprocess.csv"
df = pd.read_csv(preprocess_path)
for index, (answer, claim) in enumerate(zip(all_answer_batch_2, all_claim_batch_2)):
    if index % 200 == 0:
        print(f"Processing index {index}")
    try:
        response = await preprocessing_claim(claim, answer, llm_wrapper, llm_cache, 1024, [])
        if response is None:
            print(f"Index {index} has no response")
            continue
        else:
            df.at[index, "claim"] = response
    except Exception as e:
        print(f"Index {index} has error")
        print(e)

df.to_csv(preprocess_path, index=False)

In [ ]:
from pretty_prompt_compare import PrettyCompare

pretty_compare = PrettyCompare(compare_response=True, target=["they", "They", 
                                                              "them", "Them",
                                                              "their", "Their",
                                                              "theirs", "Theirs",
                                                              "them", "Them",
                                                              "these", "These",
                                                              "those", "Those",
                                                              "this", "This",
                                                              "that", "That",
                                                              "these", "These",
                                                              "those", "Those",
                                                              "this", "This"])

claim_processed = df.claim.tolist()
all_claim_batch_2 = dataset_loader["subtask1_batch2"].claim.tolist()

index = 21

all_claim_batch_2[index] |pretty_compare| claim_processed[index]

In [ ]:
df.at[0, "claim"] = "2"
df.head()

In [ ]:
#!/usr/bin/env python3

import os
import re
from datetime import datetime
from collections import defaultdict

def find_and_rename_newest_files_by_number():
    """
    Find the newest claim_batch2 files for each number (0-999) and rename them to claim_test format
    """
    outputs_dir = "outputs"
    
    # Pattern to match: claim_batch2_{number}_result_{timestamp}.pkl
    pattern = r'claim_batch2_(\d+)_result_(\d{8}_\d{6})\.(pkl|txt)'
    
    # Dictionary to store files by number, then by timestamp
    files_by_number = defaultdict(lambda: defaultdict(list))
    
    # Scan all files in outputs directory
    for filename in os.listdir(outputs_dir):
        match = re.match(pattern, filename)
        if match:
            number = match.group(1)
            timestamp = match.group(2)
            extension = match.group(3)
            
            files_by_number[number][timestamp].append({
                'filename': filename,
                'number': number,
                'timestamp': timestamp,
                'extension': extension,
                'full_path': os.path.join(outputs_dir, filename)
            })
    
    if not files_by_number:
        print("No files matching the pattern found!")
        return
    
    print(f"Found files for {len(files_by_number)} different numbers")
    
    # For each number, find the newest timestamp and rename those files
    renamed_count = 0
    numbers_processed = []
    
    for number in sorted(files_by_number.keys(), key=int):
        timestamps_for_number = files_by_number[number]
        
        # Find the newest timestamp for this number
        newest_timestamp = max(timestamps_for_number.keys())
        newest_files = timestamps_for_number[newest_timestamp]
        
        print(f"\nNumber {number}: Found {len(newest_files)} files with newest timestamp {newest_timestamp}")
        
        # Rename the files for this number
        for file_info in newest_files:
            old_path = file_info['full_path']
            
            # Create new filename: claim_test_result_{number}_{timestamp}.{extension}
            new_filename = f"claim_test_result_{file_info['number']}_{file_info['timestamp']}.{file_info['extension']}"
            new_path = os.path.join(outputs_dir, new_filename)
            
            try:
                os.rename(old_path, new_path)
                print(f"  Renamed: {file_info['filename']} -> {new_filename}")
                renamed_count += 1
            except Exception as e:
                print(f"  Error renaming {file_info['filename']}: {e}")
        
        numbers_processed.append(number)
    
    print(f"\nSuccessfully renamed {renamed_count} files for {len(numbers_processed)} different numbers!")
    
    # Show summary
    print(f"\nProcessed numbers: {', '.join(sorted(numbers_processed, key=int))}")
    
    # Show which numbers from 0-999 are missing
    all_numbers = set(str(i) for i in range(1000))
    found_numbers = set(files_by_number.keys())
    missing_numbers = all_numbers - found_numbers
    
    if missing_numbers:
        missing_sorted = sorted(missing_numbers, key=int)
        print(f"\nNumbers with no files found: {', '.join(missing_sorted[:20])}")
        if len(missing_numbers) > 20:
            print(f"... and {len(missing_numbers) - 20} more")


find_and_rename_newest_files_by_number()


find_and_rename_newest_files()

In [ ]:
from utils import get_latest_result_by_prefix
index  = 0
claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_result_{index}_")


In [ ]:
import os
import re

def rename_claim_test_files():
    """
    Rename files from claim_test_result_{number}_{timestamp}.{ext} 
    to claim_test_{number}_result_{timestamp}.{ext}
    """
    outputs_dir = "outputs"
    
    # Pattern to match: claim_test_result_{number}_{timestamp}.{extension}
    pattern = r'claim_test_result_(\d+)_(\d{8}_\d{6})\.(pkl|txt)'
    
    # Get all files in outputs directory
    files_to_rename = []
    
    for filename in os.listdir(outputs_dir):
        match = re.match(pattern, filename)
        if match:
            number = match.group(1)
            timestamp = match.group(2)
            extension = match.group(3)
            
            # Create new filename: claim_test_{number}_result_{timestamp}.{extension}
            new_filename = f"claim_test_{number}_result_{timestamp}.{extension}"
            
            files_to_rename.append({
                'old_filename': filename,
                'new_filename': new_filename,
                'old_path': os.path.join(outputs_dir, filename),
                'new_path': os.path.join(outputs_dir, new_filename)
            })
    
    if not files_to_rename:
        print("No files matching the pattern 'claim_test_result_*' found!")
        return
    
    print(f"Found {len(files_to_rename)} files to rename:")
    for file_info in files_to_rename:
        print(f"  {file_info['old_filename']} -> {file_info['new_filename']}")
    
    print(f"\nProceeding with renaming {len(files_to_rename)} files...")
    
    # Perform the renaming
    renamed_count = 0
    for file_info in files_to_rename:
        try:
            os.rename(file_info['old_path'], file_info['new_path'])
            print(f"✓ Renamed: {file_info['old_filename']} -> {file_info['new_filename']}")
            renamed_count += 1
        except Exception as e:
            print(f"✗ Error renaming {file_info['old_filename']}: {e}")
    
    print(f"\nSuccessfully renamed {renamed_count} out of {len(files_to_rename)} files!")

rename_claim_test_files()

In [ ]:
from utils import get_latest_result_by_prefix

index = 1
claim_nodes, claim_edges = get_latest_result_by_prefix(f"claim_test_{index}_")

claim_nodes

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import numpy as np
from kg_compare import get_src_tgt_dict, get_map_src_tgt_dict, find_best_span

from kg_compare import compare_knowledge_graphs_similarity, find_best_span_for_all_evidences
from utils import get_latest_result_by_prefix
import seaborn as sns

import numpy as np
from sklearn.cluster import KMeans
import os
all_responses = []

import pandas as pd

from geminillm import gemini_complete_if_cache
from json_kv_iml import JsonKVStorage
from operationCheatSheet import preprocessing_claim
from openaillm import openai_embed

In [3]:
LLM_MODEL_NAME = "gemini-2.5-flash-preview-05-20"
def write_prediction_to_file(index, row_id, prediction, output_file="outputs/response_test.txt"):
    """
    Write a line with row ID and prediction to the output file
    
    Args:
        row_id: The ID/index of the row being processed
        prediction: The prediction result
        output_file: Path to the output file
    """
    # Ensure outputs directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Write the prediction with row ID to file
    with open(output_file, 'a', encoding='utf-8') as f:
        f.write(f"{index}, {row_id}, {prediction}\n")

async def llm_wrapper(prompt, history_messages=None, max_tokens=None, **kwargs):
    if history_messages is None:
        history_messages = []


    # Use Google GenAI
    return await gemini_complete_if_cache(
        model=LLM_MODEL_NAME,
        prompt=prompt,
        history_messages=history_messages,
        hashing_kv=llm_cache,
        temperature=0.2,
        max_tokens=max_tokens or 1024,
    )

In [4]:
TASKNAMES = ["subtask1", "subtask2"]
TYPES = ["test", "test_preprocess", "train_batch2", "train_batch2_preprocess"]

dataset_loader = dict()

for taskname in TASKNAMES:
    for type in TYPES:
        print(f"Processing {taskname} for {type}")
        prefix = "dataset"
        # Load the data
        
        if taskname == "subtask2" and type == "test_preprocess":
            continue
        if taskname == "subtask2" and type == "train_batch2_preprocess":
            continue
        data = pd.read_csv(f"{prefix}/{taskname}_{type}.csv")
        dataset_loader[f"{taskname}_{type}"] = data

dataset_loader["subtask1_test"]["claim"] = dataset_loader["subtask1_test_preprocess"]["claim"]
dataset_loader["subtask1_train_batch2"]["claim"] = dataset_loader["subtask1_train_batch2_preprocess"]["claim"]

output_file = "outputs/response_test.txt"
os.makedirs(os.path.dirname(output_file), exist_ok=True)

Processing subtask1 for test
Processing subtask1 for test_preprocess
Processing subtask1 for train_batch2
Processing subtask1 for train_batch2_preprocess
Processing subtask2 for test
Processing subtask2 for test_preprocess
Processing subtask2 for train_batch2
Processing subtask2 for train_batch2_preprocess


In [5]:
test_data = dataset_loader["subtask1_test"]

pickle_prefix = "test"
# pickle_prefix = "batch2"

for index in range(len(test_data)):
    if index % 50 == 0:
        print(f"Processing index {index}")
    
    row_id = test_data.at[index, "ID"]
    claim_content = test_data.at[index, "claim"]

    claim_nodes, claim_edges = await get_latest_result_by_prefix(f"claim_{pickle_prefix}_{index}_")
    reference_nodes, reference_edges = await get_latest_result_by_prefix(f"reference_{pickle_prefix}_{index}_")
    reference_content = test_data.at[index, "reference"]

    claim_node_list = list(claim_nodes.keys())
    reference_node_list = list(reference_nodes.keys())

    if claim_node_list == [] or reference_node_list == []:
        all_responses.append("Unverifiable")
        write_prediction_to_file(index, row_id, "Unverifiable")
        continue
        
    similarity_matrix = compare_knowledge_graphs_similarity(claim_node_list,
                                                            reference_node_list,
                                                            if_plot=False)
    src_tgt_dict = get_src_tgt_dict(similarity_matrix, claim_node_list, reference_node_list)

    claim_reference_edge_map = get_map_src_tgt_dict(src_tgt_dict, claim_edges)

    evidence_text = find_best_span_for_all_evidences(reference_edges,
                                 claim_reference_edge_map,
                                 reference_content)
    
    if evidence_text == "":
        evidence_text = reference_content
    
    classification_prompt = f"""
    You are an expert hallucination classifier. Your task is to classify a claim based solely on the provided evidence into one of the following three categories:

    - Entailment
    - Contradiction
    - Unverifiable

    Claim:
    {claim_content}

    Evidence:
    {evidence_text}

    Instructions:
    - You MUST respond with ONLY one of the three words: "Entailment", "Contradiction", or "Unverifiable"
    - Respond "Unverifiable" if the evidence is irrelevant, insufficient, or cannot confirm or deny the claim
    - Respond "Contradiction" if the claim contradicts, misinterprets, misleads, or overstates the evidence
    - Respond "Entailment" if the evidence reasonably supports the claim, even if some uncertainty exists
    - Do NOT provide explanations, justifications, or any additional text
    - Your entire response must be EXACTLY one of these three words: "Entailment", "Contradiction", or "Unverifiable"
    """

    response = await llm_wrapper(classification_prompt)
    all_responses.append(response)
    write_prediction_to_file(index, row_id, response)
    
    

Processing index 0
Loading latest result for prefix 'claim_test_0_': claim_test_0_result_20250622_190433.pkl
Loading latest result for prefix 'reference_test_0_': reference_test_0_result_20250623_133422.pkl
Loading latest result for prefix 'claim_test_1_': claim_test_1_result_20250622_190508.pkl
Loading latest result for prefix 'reference_test_1_': reference_test_1_result_20250623_133526.pkl
Loading latest result for prefix 'claim_test_2_': claim_test_2_result_20250622_190558.pkl
Loading latest result for prefix 'reference_test_2_': reference_test_2_result_20250623_133728.pkl
Loading latest result for prefix 'claim_test_3_': claim_test_3_result_20250622_190629.pkl
Loading latest result for prefix 'reference_test_3_': reference_test_3_result_20250623_133856.pkl
Loading latest result for prefix 'claim_test_4_': claim_test_4_result_20250622_190708.pkl
Loading latest result for prefix 'reference_test_4_': reference_test_4_result_20250623_134004.pkl
Loading latest result for prefix 'claim_t

In [8]:
import pandas as pd

# Read the original file
with open("outputs/response_test.txt", "r") as f:
    lines = f.readlines()

# Split each line and extract ID and label
full_data = [line.strip().split(", ")[1:] for line in lines]  # skip the first field (index)

# Create DataFrame
df = pd.DataFrame(full_data, columns=["ID", "label"])
df["label"] = df["label"].replace({
    "Unverifiable": "unver",
    "Contradiction": "contra",
    "Entailment": "entail"
})

# Drop exact duplicate rows (same ID and label)
df = df.drop_duplicates()

df.to_csv("outputs/kaggle_results/response_clean.csv", index=False)

In [7]:
df

,ID,label
0,i_1954,unver
1,s_1594,unver
2,i_2194,contra
3,s_1906,contra
4,i_95,contra
...,...,...
995,s_339,unver
996,i_1736,entail
997,i_493,unver
998,s_743,unver


In [25]:
results_df = dataset_loader["subtask1_train_batch2"][:100].copy()
results_df["true_label"] = results_df["label"]
results_df["predicted_label"]=  df.label[-100:].values
results_df

,ID,question,answer,claim,reference,label,justification,true_label,predicted_label
0,s_4,a chatbot enhanced with retrieval capabilities...,** Chatbot Enhanced with Retrieval Capabilitie...,Virtual Assistants are limited in supporting c...,[3]: The widespread use of chatbots is a reali...,contra,The claim directly contradicts the reference b...,contra,unver
1,s_5,a chatbot enhanced with retrieval capabilities...,** Chatbot Enhanced with Retrieval Capabilitie...,Educational Benefits and Applications: Student...,[4]: Education is one of the important factors...,contra,"The claim overstates the statement ""Chatbots m...",contra,unver
2,s_7,a chatbot enhanced with retrieval capabilities...,** Chatbot Enhanced with Retrieval Capabilitie...,"Chatbots should offer responsive interactions,...",[6]: Purpose: Chatbots have been widely adopte...,contra,The claim misinterprets the reference by stati...,contra,contra
3,s_8,a chatbot enhanced with retrieval capabilities...,** Chatbot Enhanced with Retrieval Capabilitie...,Chatbots can use natural language processing t...,[4]: Education is one of the important factors...,unver,"The claim states that ""chatbots might not nece...",unver,unver
4,s_9,a chatbot enhanced with retrieval capabilities...,** Chatbot Enhanced with Retrieval Capabilitie...,Challenges and Considerations: Technological a...,[3]: The widespread use of chatbots is a reali...,contra,The claim contradicts with the reference by st...,contra,contra
...,...,...,...,...,...,...,...,...,...
95,s_185,What specific security issues arise from incor...,To address the query about specific security i...,Unauthorized Access and Data Breaches: IoT-con...,[1]: The Internet of Things (IoT) enables us t...,entail,The claim is explicitly supported by the refer...,entail,entail
96,s_187,What specific security issues arise from incor...,To address the query about specific security i...,Vulnerability to Cyber Attacks: IoT devices ar...,[4]: Internet of Things (IoT) devices present ...,entail,A reference [4] supports the claim completely.,entail,entail
97,s_191,What types of interactions are possible with a...,** Types of Interactions with Autonomous Mobil...,Social Interactions: social robots can identif...,[1]: The increasing use of autonomous mobile r...,entail,The claim is supported by the given reference.,entail,unver
98,s_192,What types of interactions are possible with a...,** Types of Interactions with Autonomous Mobil...,Human-Robot Interaction (HRI): Robots are inef...,[2]: As a testbed for real-world experimentati...,contra,The claim misinterprets the reference by menti...,contra,unver


In [26]:
from sklearn.metrics import f1_score, classification_report


# Calculate evaluation metrics
true_labels = results_df['true_label'].tolist()
predicted_labels = results_df['predicted_label'].tolist()

# Calculate weighted F1 score (main evaluation metric)
weighted_f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f"\n=== EVALUATION RESULTS ===")
print(f"Total samples processed: {len(results_df)}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")

# Calculate accuracy
accuracy = (results_df['true_label'] == results_df['predicted_label']).mean()
print(f"Accuracy: {accuracy:.4f}")

# Print detailed classification report
print(f"\n=== DETAILED CLASSIFICATION REPORT ===")
print(classification_report(true_labels, predicted_labels))

# Show results by batch
print(f"\n=== RESULTS BY BATCH ===")
for batch in results_df['batch'].unique():
    batch_results = results_df[results_df['batch'] == batch]
    batch_accuracy = (batch_results['true_label'] == batch_results['predicted_label']).mean()
    batch_f1 = f1_score(batch_results['true_label'], batch_results['predicted_label'], average='weighted')
    print(f"{batch}: Accuracy={batch_accuracy:.4f}, Weighted F1={batch_f1:.4f}")

# Calculate F1 scores for batch2 and batch3 separately
print(f"\n=== INDIVIDUAL BATCH F1 SCORES ===")
if 'batch2' in results_df['batch'].values:
    batch2_results = results_df[results_df['batch'] == 'batch2']
    batch2_f1 = f1_score(batch2_results['true_label'], batch2_results['predicted_label'], average='weighted')
    print(f"Batch2 Weighted F1 Score: {batch2_f1:.4f}")

if 'batch3' in results_df['batch'].values:
    batch3_results = results_df[results_df['batch'] == 'batch3']
    batch3_f1 = f1_score(batch3_results['true_label'], batch3_results['predicted_label'], average='weighted')
    print(f"Batch3 Weighted F1 Score: {batch3_f1:.4f}")

# Display first few results
print(f"\n=== SAMPLE RESULTS ===")
print(results_df.head(10))



=== EVALUATION RESULTS ===
Total samples processed: 100
Weighted F1 Score: 0.5714
Accuracy: 0.5600

=== DETAILED CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

      contra       0.76      0.54      0.63        35
      entail       1.00      0.38      0.56        39
       unver       0.37      0.85      0.51        26

    accuracy                           0.56       100
   macro avg       0.71      0.59      0.57       100
weighted avg       0.75      0.56      0.57       100


=== RESULTS BY BATCH ===


KeyError: 'batch'

In [ ]:
set(df.index.unique())